In [1]:
import numpy as np # linear algebra
np.random.seed(10)
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Required magic to display matplotlib plots in notebooks
%matplotlib inline

from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("trisha2094/weatheraus")

print("Path to dataset files:", path)

Path to dataset files: /Users/lucia/.cache/kagglehub/datasets/trisha2094/weatheraus/versions/1


In [3]:
data = '/Users/lucia/.cache/kagglehub/datasets/trisha2094/weatheraus/versions/1/weatherAUS.csv'

df = pd.read_csv(data)

In [4]:
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,0.0,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,0.0,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,0.0,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,1.0,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,0.2,No


In [5]:
df.columns

Index(['Date', 'Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation',
       'Sunshine', 'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm',
       'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am',
       'Temp3pm', 'RainToday', 'RISK_MM', 'RainTomorrow'],
      dtype='object')

In [6]:
y = df["RainTomorrow"]  # Variable objetivo
X = df.drop(columns=["RainTomorrow"])  # Resto de variables como features

# Mostrar las dimensiones para verificar
print(f"Shape of X: {X.shape}")
print(f"Shape of y: {y.shape}")

Shape of X: (142193, 23)
Shape of y: (142193,)


In [7]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [8]:
X_train = X_train.drop(columns=["Date"])
X_test = X_test.drop(columns=["Date"])


In [9]:
# Import necessary libraries
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [10]:
print(X_train.dtypes)


Location          object
MinTemp          float64
MaxTemp          float64
Rainfall         float64
Evaporation      float64
Sunshine         float64
WindGustDir       object
WindGustSpeed    float64
WindDir9am        object
WindDir3pm        object
WindSpeed9am     float64
WindSpeed3pm     float64
Humidity9am      float64
Humidity3pm      float64
Pressure9am      float64
Pressure3pm      float64
Cloud9am         float64
Cloud3pm         float64
Temp9am          float64
Temp3pm          float64
RainToday         object
RISK_MM          float64
dtype: object


In [11]:
# Convertir variables categóricas en variables numéricas
X_train = pd.get_dummies(X_train, columns=["Location", "WindGustDir", "WindDir9am", "WindDir3pm", "RainToday"], drop_first=True)
X_test = pd.get_dummies(X_test, columns=["Location", "WindGustDir", "WindDir9am", "WindDir3pm", "RainToday"], drop_first=True)

# Asegurar que las columnas de train y test coincidan después de dummies
X_train, X_test = X_train.align(X_test, join="left", axis=1, fill_value=0)


In [12]:
from sklearn.impute import SimpleImputer  # Importar la clase

# Aplicar imputación
imputer = SimpleImputer(strategy="mean")  # También puedes usar "median" o "most_frequent"
X_train_array = imputer.fit_transform(X_train)  # Devuelve un numpy.ndarray
X_test_array = imputer.transform(X_test)

# Convertir de nuevo a DataFrame con las columnas originales
X_train = pd.DataFrame(X_train_array, columns=X_train.columns)
X_test = pd.DataFrame(X_test_array, columns=X_test.columns)


In [13]:
!pip install imbalanced-learn


In [14]:
from imblearn.over_sampling import SMOTE

# Aplicar SMOTE para balancear las clases
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

# Verificar balance de clases
print(y_train_balanced.value_counts())

# Entrenar el modelo con datos balanceados
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42)
mlp.fit(X_train_balanced, y_train_balanced)


No     88218
Yes    88218
Name: RainTomorrow, dtype: int64


MLPClassifier(max_iter=500, random_state=42)

In [15]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_balanced)
X_test = scaler.transform(X_test)


In [16]:

# Make predictions on the test data
y_pred = mlp.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.90


/Users/lucia/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


In [17]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)
print(cm)


[[22097     1]
 [ 2979  3362]]


In [19]:
from sklearn.metrics import recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder

# Convertir 'Yes' y 'No' a 1 y 0
encoder = LabelEncoder()
y_test_numeric = encoder.fit_transform(y_test)
y_pred_numeric = encoder.transform(y_pred)

# Sensibilidad (Recall)
recall = recall_score(y_test_numeric, y_pred_numeric)
print(f"Sensibilidad (Recall): {recall:.2f}")

# F1-score
f1 = f1_score(y_test_numeric, y_pred_numeric)
print(f"F1-score: {f1:.2f}")

# AUC-ROC (requiere probabilidades en vez de clases)
y_prob = mlp.predict_proba(X_test)[:, 1]  # Probabilidad de la clase positiva
auc = roc_auc_score(y_test_numeric, y_prob)
print(f"AUC-ROC: {auc:.2f}")



Sensibilidad (Recall): 0.53
F1-score: 0.69
AUC-ROC: 0.94


/Users/lucia/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
